In [ ]:
""" msg to text """
import extract_msg

# Path to your .msg file
file_path = "ahc_invoice.msg"

# Open and read the .msg file
msg = extract_msg.Message(file_path)



""" Extract attachments (if any) """
if msg.attachments:
    print("Attachments:")
    for attachment in msg.attachments:
        print(attachment.longFilename)





""" Extract attachments from .msg file """

import extract_msg
import os

output_folder = "case_files"  # Folder to save attachments


file_path = "ahc_invoice.msg"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Open the .msg file
msg = extract_msg.Message(file_path)

# Extract attachments
if msg.attachments:
    print("Attachments found:")
    for attachment in msg.attachments:
        attachment_path = os.path.join(output_folder, attachment.longFilename)
        with open(attachment_path, "wb") as f:
            f.write(attachment.data)
        print(f"Saved: {attachment_path}")
else:
    print("No attachments found.")


In [ ]:
import os
import pathlib
from supabase import create_client
from dotenv import load_dotenv
import magic
from typing import Optional

def get_file_type(file_path: str) -> Optional[str]:
    mime = magic.Magic()
    return mime.from_file(file_path).split('/')[0].capitalize()

async def process_directory(base_path: str, case_id: str, user_id: str, lead_ip: str):
    """Recursively process directory and upload metadata to Supabase"""
    supabase = create_client(
        os.getenv("SUPABASE_URL"),
        os.getenv("SUPABASE_KEY")
    )
    
    for root, _, files in os.walk(base_path):
        for filename in files:
            file_path = pathlib.Path(root) / filename
            try:
                # LLM integration point for title generation
                document_title = await generate_document_title(file_path)  # Need LLM details
                
                record = {
                    "case_id": case_id,
                    "user_id": user_id,
                    "document_title": document_title,
                    "document_file_name": filename,
                    "document_type": get_file_type(str(file_path)),
                    "file_path": str(file_path.relative_to(base_path)),
                    "file_size": file_path.stat().st_size,
                    "uploaded_by": lead_ip
                }
                
                # Check for existing record
                existing = supabase.table('case_documents') \
                    .select('document_id') \
                    .eq('file_path', record['file_path']) \
                    .execute()
                
                if not existing.data:
                    supabase.table('case_documents').insert(record).execute()
                    
            except Exception as e:
                logging.error(f"Error processing {file_path}: {str(e)}")

Found rules: items {
  sip_dispatch_rule_id: "SDR_9oiituE5jeyG"
  rule {
    dispatch_rule_individual {
      room_prefix: "call-_+447723180506-"
    }
  }
  trunk_ids: "ST_2CrNutUodeAF"
  name: "rule_+447723180506_ST_2CrNutUodeAF"
}

Deleted rule with ID: SDR_9oiituE5jeyG
